# eXtreme Gradient Boosting (XGB) Model, Trained Globally

In [19]:
# Standard imports
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import pickle

# Machine learning libraries
import sklearn            # machine-learning libary with many algorithms implemented
import xgboost as xgb     # extreme gradient boosting (XGB)
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, load_model

# Python file with supporting functions
import model_utils

In [20]:
global_model_path = '/home/julias/MLEE-final-project/models/saved_models/global_models'

### To Set and Track Seeds for Reproducibility (Referencing Group Standard):

In [21]:
ens = 'CESM'
member = '009'
reference_output_dir = '/home/julias/MLEE-final-project/pickle_files'

path_seeds = f'{reference_output_dir}/random_seeds.npy'
random_seeds = np.load(path_seeds)  

path_loc = f'{reference_output_dir}/cesm_seed_loc_dict.pickle'
with open(path_loc,'rb') as handle:
    seed_loc_dict = pickle.load(handle)
seed_loc = seed_loc_dict[ens][member]

# for next project, where train and test datasets use all CESM members
#path_cesm = f"{reference_output_dir}/cesm_members_dict.pickle"
#with open(path_cesm,'rb') as handle:
#    cesm_mems_dict = pickle.load(handle)

In [22]:
# random_seeds

In [23]:
# seed_loc_dict # using CESM 009, so seed_loc should be 32

In [24]:
# seed_loc #confirmed 32

# Load Split Datasets and Create Versions for Experimentation

## Load Split Datasets

In [25]:
X_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/X.nc').to_dataframe().dropna()
y_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/y.nc').to_dataframe().dropna() 
X_train_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/X_train.nc').to_dataframe().dropna() 
y_train_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/y_train.nc').to_dataframe().dropna()
X_test_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/X_test.nc').to_dataframe().dropna() 
y_test_df = xr.open_dataset('/home/julias/MLEE-final-project/proc_data/global_split_datasets/y_test.nc').to_dataframe().dropna() 

Check that data was saved and loaded properly:

In [26]:
X_train_df

SSS       SST         MLD       Chl  \
xlon   ylat  time                                                    
-179.5 -77.5 1982-03-15  33.997498  0.640288   26.234657  0.331255   
             1982-04-15  34.180756 -1.755514   59.944748  0.641632   
             1982-05-15  34.445652 -1.776272  220.441910  0.057961   
             1982-06-15  34.508339 -1.809904  381.795532  0.017642   
             1982-08-15  34.562683 -1.823900  429.841278  0.005047   
...                            ...       ...         ...       ...   
 179.5  62.5 1996-01-15  32.590897 -1.597806   27.985397  0.074352   
             1998-12-15  32.283058  0.631731   25.075340  0.897956   
             1999-12-15  32.390461  0.080004   26.077244  0.703162   
             2010-02-15  32.757904 -1.240300   27.023640  0.085494   
             2011-12-15  32.434753  0.346877   24.999208  0.672751   

                               XCO2        T0        T1         A         B  \
xlon   ylat  time                                                             
-179.5 -77.5 1982-03-15  340.962250  0.292600  0.956235 -0.976296 -0.001889   
             1982-04-15  341.075439 -0.234491  0.972118 -0.976296 -0.001889   
             1982-05-15  341.193176 -0.683919  0.729558 -0.976296 -0.001889   
             1982-06-15  341.310730 -0.959933  0.280231 -0.976296 -0.001889   
             1982-08-15  341.545685 -0.720667 -0.693281 -0.976296 -0.001889   
...                             ...       ...       ...       ...       ...   
 179.5  62.5 1996-01-15  361.389465  0.966848  0.255353  0.887011  0.004029   
             1998-12-15  367.086853  0.962309 -0.271958  0.887011  0.004029   
             1999-12-15  368.669678  0.962309 -0.271958  0.887011  0.004029   
             2010-02-15  389.416168  0.702527  0.711657  0.887011  0.004029   
             2011-12-15  393.703278  0.962309 -0.271958  0.887011  0.004029   

                                C  
xlon   ylat  time                  
-179.5 -77.5 1982-03-15  0.216431  
             1982-04-15  0.216431  
             1982-05-15  0.216431  
             1982-06-15  0.216431  
             1982-08-15  0.216431  
...                           ...  
 179.5  62.5 1996-01-15  0.461731  
             1998-12-15  0.461731  
             1999-12-15  0.461731  
             2010-02-15  0.461731  
             2011-12-15  0.461731  

[11865574 rows x 10 columns]

In [27]:
y_test_df

pCO2
xlon   ylat  time                  
-179.5 -77.5 1982-02-15  165.219524
             1982-07-15  332.735048
             1982-12-15  150.877219
             1983-05-15  311.177084
             1983-10-15  338.556662
...                             ...
 179.5  62.5 1985-01-15  380.706079
             1987-02-15  391.417257
             1990-01-15  431.635894
             1992-12-15  406.267746
             2010-01-15  468.311163

[2966454 rows x 1 columns]

### Create Numpy Arrays for Original Data

In [43]:
X_original = X_df.to_numpy()         
y_original = y_df.to_numpy().ravel() 
X_train_original = X_train_df.to_numpy() 
y_train_original = y_train_df.to_numpy().ravel()
X_test_original = X_test_df.to_numpy()  
y_test_original = y_test_df.to_numpy().ravel()

## Create Normalized Dataframes

*Note: Done here, as opposed to in test/train split, so that I can save the original train/test datasets and later determine whether normalization led to improvement. (My group has not historically normalized data before training.)*

In [ ]:
#X_df_norm = (X_df - X_df.mean())/X_df.std() 
#y_df_norm = (y_df - y_df.mean())/y_df.std()
#X_train_df_norm = (X_train_df - X_train_df.mean())/X_train_df.std()
#y_train_df_norm = (y_train_df - y_train_df.mean())/y_train_df.std()
#X_test_df_norm = (X_test_df - X_test_df.mean())/X_test_df.std()
#y_test_df_norm = (y_test_df - y_test_df.mean())/y_test_df.std()

CHANGE METHOD TO BELOW
- This way, time and lat/lon conversions are not normalized
- Note that coordinates don't seem to be input into ML algorithms, so T0,T1,A,B,C are the inputs of time and space (as would want, don't want two forms of time and space input)

In [49]:
X_df_norm = X_df
y_df_norm = y_df
X_train_df_norm = X_train_df
y_train_df_norm = y_train_df
X_test_df_norm = X_test_df
y_test_df_norm = y_test_df

for df in X_df_norm, X_train_df_norm, X_test_df_norm:
    df.SSS = (df.SSS - df.SSS.mean())/df.SSS.std() 
    df.SST = (df.SST - df.SST.mean())/df.SST.std() 
    df.MLD = (df.MLD - df.MLD.mean())/df.MLD.std() 
    df.Chl = (df.Chl - df.Chl.mean())/df.Chl.std() 
    df.XCO2 = (df.XCO2 - df.XCO2.mean())/df.XCO2.std() 

for df in y_df_norm, y_train_df_norm, y_test_df_norm:
    df.pCO2 = (df.pCO2 - df.pCO2.mean())/df.pCO2.std() 

In [51]:
X_train_df_norm

SSS       SST       MLD       Chl      XCO2  \
xlon   ylat  time                                                           
-179.5 -77.5 1982-03-15 -0.265890 -1.488140 -0.692089  0.127524 -1.566749   
             1982-04-15 -0.102081 -1.720083 -0.150905  0.678477 -1.560679   
             1982-05-15  0.134703 -1.722093  2.425728 -0.357605 -1.554364   
             1982-06-15  0.190737 -1.725349  5.016110 -0.429174 -1.548060   
             1982-08-15  0.239313 -1.726704  5.787440 -0.451532 -1.535459   
...                           ...       ...       ...       ...       ...   
 179.5  62.5 1996-01-15 -1.523210 -1.704815 -0.663983 -0.328508 -0.471215   
             1998-12-15 -1.798378 -1.488968 -0.710701  1.133481 -0.165658   
             1999-12-15 -1.702374 -1.542382 -0.694616  0.787700 -0.080769   
             2010-02-15 -1.373927 -1.670204 -0.679423 -0.308730  1.031888   
             2011-12-15 -1.662782 -1.516546 -0.711923  0.733718  1.261811   

                               T0        T1         A         B         C  
xlon   ylat  time                                                          
-179.5 -77.5 1982-03-15  0.292600  0.956235 -0.976296 -0.001889  0.216431  
             1982-04-15 -0.234491  0.972118 -0.976296 -0.001889  0.216431  
             1982-05-15 -0.683919  0.729558 -0.976296 -0.001889  0.216431  
             1982-06-15 -0.959933  0.280231 -0.976296 -0.001889  0.216431  
             1982-08-15 -0.720667 -0.693281 -0.976296 -0.001889  0.216431  
...                           ...       ...       ...       ...       ...  
 179.5  62.5 1996-01-15  0.966848  0.255353  0.887011  0.004029  0.461731  
             1998-12-15  0.962309 -0.271958  0.887011  0.004029  0.461731  
             1999-12-15  0.962309 -0.271958  0.887011  0.004029  0.461731  
             2010-02-15  0.702527  0.711657  0.887011  0.004029  0.461731  
             2011-12-15  0.962309 -0.271958  0.887011  0.004029  0.461731  

[11865574 rows x 10 columns]

## Create Numpy Arrays 

In [ ]:
X_n = X_df_norm.to_numpy()         
y_n = y_df_norm.to_numpy().ravel() 
X_train_n = X_train_df_norm.to_numpy()
y_train_n = y_train_df_norm.to_numpy().ravel()
X_test_n = X_test_df_norm.to_numpy() 
y_test_n = y_test_df_norm.to_numpy().ravel()

# Select Approach & Parameters

## Approach

Based on preliminary experimentation in the non-global XGB notebook (XGB.ipynb) will only train the global dataset on the identified ideal approach. Thus, will train on dataframes (as opposed to numpy arrays) and normalized data.

## Parameters

### Reference Best Parameters from Previous Group Work

Published in Bennington 2022, trained XGB to learn pCO2 residual (pC02 change with direct temperature effects removed)

In [ ]:
path_bp='/data/artemis/workspace/vbennington/full_sst/pCO2_DIC/models/performance_metrics/xg/xg_best_params_dict.pickle'
with open(path_bp,'rb') as handle:
    best_params = pickle.load(handle)
print(best_params)

Use previous CESM best parameters as a starting point:
- max_depth = 6
- n_estimators = 4000

### Investigate Ideal Parameters for Selected Approach

- Test with three different n_estimators and 3 different depths
- Advice from group post-doc: For XGB, too many depth layers may lead to overfitting (usually 8 or less layers for XGB)
- We want the combo of xg_param_grid that gives the lowest RMSE

- Was not able to use GridSearchCV (allowed to run for hours but kernel crashed on multiple attempts. 
- Will re-attempt on a future project using NCAR supercomputer account once have access.

In [ ]:
# best_params = {}
# model = xgb.XGBRegressor(verbosity=0) 
# xg_param_grid = {'n_estimators':[3000, 4000, 5000],
#                  'max_depth':[5, 6, 7]}
# grid = GridSearchCV(model, xg_param_grid, scoring='neg_mean_squared_error') #, cv=K_folds, return_train_score=False, refit=False

In [ ]:
# grid.fit(X_train_df_norm, y_train_df_norm)

In [ ]:
# best_params[ens] = grid.best_params_
# print(best_params)

# Preliminary XGB Models

***Goal: Compare auto-XGB with lowest end and highest end of group XGB parameter matrix***

## Baseline Model Test

Alternations should at least improve on the baseline set by the auto-generated XGB algorithm

In [ ]:
model_baseline = xgb.XGBRegressor(verbosity=0) 

In [ ]:
model_baseline.fit(X_train_df_norm, y_train_df_norm)

Learning rate looks high, may need to manually set

In [ ]:
train_score_baseline = model_baseline.score(X_train_df_norm, y_train_df_norm) 
train_score_baseline

In [ ]:
test_score_baseline = model_baseline.score(X_test_df_norm, y_test_df_norm) 
test_score_baseline

In [ ]:
y_pred_baseline = model_baseline.predict(X_test_df_norm)
mse = mean_squared_error(y_test_df_norm, y_pred_baseline)
print("MSE: %.2f" % mse)

In [ ]:
x_ax = range(len(y_test_df))
plt.plot(x_ax, y_test_df_norm, label="original")
plt.plot(x_ax, y_pred_baseline, label="predicted")
plt.title("pC02 test and predicted data")
plt.legend()
plt.show()

In [ ]:
y_test_df

In [ ]:
#y_test_df_norm = (y_test_df - y_test_df.mean())/y_test_df.std()
# y_baseline = y_pred_baseline * y_test_df.std() + y_test_df.mean()

In [ ]:
y_test_df_norm['baseline_XGB_pred'] = y_pred_baseline

In [ ]:
y_test_df_norm.baseline_XGB_pred

In [ ]:
y_test_df = y_test_df.pCO2

In [ ]:
y_test_df

In [ ]:
model_baseline.save_model(os.path.join(global_model_path,'XGB_prelim_baseline.h5'))

In [ ]:
model_test = load_model(os.path.join(global_model_path,'XGB_prelim_test.json'))

## Lowest Combination of XGB Parameter Matrix

In [44]:
model_low = XGBRegressor(n_estimators=3000, max_depth=5)
model_low.fit(X_train, y_train) 

KeyboardInterrupt: 

In [ ]:
train_score_low = model_low.score(X_train_df_norm, y_train_df_norm) 
train_score_low

In [ ]:
test_score_low = model_low.score(X_test_df_norm, y_test_df_norm) 
test_score_low

In [ ]:
y_pred_low = model_low.predict(X_test_df_norm)
mse = mean_squared_error(y_test_df_norm, y_pred_low)
print("MSE: %.2f" % mse)

In [ ]:
x_ax = range(len(y_test_df))
plt.plot(x_ax, y_test_df_norm, label="original")
plt.plot(x_ax, y_pred_low, label="predicted")
plt.title("pC02 test and predicted data")
plt.legend()
plt.show()

In [ ]:
model_low.save_model(os.path.join(global_model_path,'XGB_prelim_lowest_param2.h5'))

In [ ]:
model_low = load_model(os.path.join(global_model_path,'XGB_prelim_highest_param-Copy1.json'))

## Highest Combination of XGB Parameter Matrix

In [ ]:
model_high = XGBRegressor(n_estimators=3000, max_depth=5)
model_high.fit(X_train_df_norm, y_train_df_norm) 

In [ ]:
train_score_high = model_high.score(X_train_df_norm, y_train_df_norm) 
train_score_high

In [ ]:
test_score_high = model_high.score(X_test_df_norm, y_test_df_norm) 
test_score_high

In [ ]:
y_pred_high = model_high.predict(X_test_df_norm)
mse = mean_squared_error(y_test_df_norm, y_pred_high)
print("MSE: %.2f" % mse)

In [ ]:
x_ax = range(len(y_test_df))
plt.plot(x_ax, y_test_df_norm, label="original")
plt.plot(x_ax, y_pred_high, label="predicted")
plt.title("pC02 test and predicted data")
plt.legend()
plt.show()

In [ ]:
model_high.save_model(os.path.join(global_model_path,'XGB_prelim_highest_param.h5'))

# XGB Model

## Build XGB Model 
(based on best parameters found in previous section)

## Train XGB Model

## Save XGB Model

## Test XBG Model

# For Future Work

## Additional Validation Split 

*Note: Group does not use this method when test years are used (as I did in processed_data_split.ipynb)*

### Group's Version

#### Train/validate/test split proportions

In [ ]:
# 20% of dataset for validation, 20% of dataset for testing, the rest for training
# Training set will be split into validation and another training set

val_prop = .2
test_prop = .2

In [ ]:
# Splitting the training data into validation and another training set   
N = X_train.shape[0]
train_val_idx, train_idx, val_idx, test_idx = model_utils.train_val_test_split(N, test_prop, val_prop, random_seeds, seed_loc)
X_train_val, X_train, X_val, X_test_tmp, y_train_val, y_train, y_val, y_test_tmp = model_utils.apply_splits(X_train, y_train, train_val_idx, train_idx, val_idx, test_idx) 

## Starting New Version

In [ ]:
X_minitest = X_df #.to_numpy()
y_minitest = y_df #.to_numpy()

In [ ]:
xtrain_minitest, xtest_minitest, ytrain_minitest, ytest_minitest=train_test_split(X_minitest, y_minitest, test_size=0.15)

In [ ]:
xtrain_minitest.max()

In [ ]:
xtrain_minitest

In [ ]:
XGB_model_minitest = xgb.XGBRegressor(verbosity=0) 

In [ ]:
XGB_model_minitest.fit(xtrain_minitest, ytrain_minitest)

# eXtreme Gradient Boosting 

(to compare performance to other methods from group)

In [ ]:
# For next step of testing new hyperparameters, clear best_params
# best_params = {}  # opened above

In [ ]:
# A, B and C represent lon and lat (3 components of the n-vector; so that the algorithm doesn't interpret 0 and 360
# degrees to be far apart 
# T0 and T1 represent time

#features_sel = ['SSS','SST','MLD','Chl','XCO2','T0', 'T1','A', 'B', 'C'] 
#target_sel = ['pCO2'] 

## Building and Training the XGB Model

In [ ]:
# Number of cores you have access to for model training, group standard:
jobs = 30

### Create two dictionaries

In [ ]:
test_performance = defaultdict(dict)
unseen_performance = defaultdict(dict)

### XGB-Specific Inputs

In [ ]:
# K_folds: cross validation; number of splits for training set (in this case 3 splits; see below)
# Train on the first split, test on the remaining 2. Total 3 numbers for the final RMSE

K_folds = 3       # Split training set into 3 parts
approach = "xg"   # XGB approach
#first_mem = False # Initialize if using gridsearch to find best_params

In [ ]:
# model: define which approach to use
# param_grid: the n_estimators (decision trees) and depths
# GridSearchCV: applying the K-fold cross validation
# first_mem = False: checks only the best params for first_mem (and not all members). For the next members, the  
# parameters from first_mem are re-used
# you could try to find the best params for a few members (but not all of them)
# 9 possible combinations (3 different n_estimators, 3 different max depths) x 3 (K-fold; 3 training sets)
if first_mem:
            model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
            param_grid = xg_param_grid
            grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=K_folds, return_train_score=False, refit=False)
            grid.fit(X_train_val, y_train_val)
            best_params[ens] = grid.best_params_
            print(best_params)
            first_mem = False

In [ ]:
model = XGBRegressor(random_state=random_seeds[4,seed_loc], n_jobs=jobs)
param_grid = xg_param_grid
grid = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=K_folds, return_train_score=False, refit=False)
grid.fit(X_train, y_train)
best_params[ens] = grid.best_params_
print(best_params)

### Train the XBG model

In [ ]:
model = XGBRegressor(random_state=random_seeds[5,seed_loc], **best_params[ens], n_jobs=jobs)
model.fit(X_train_val, y_train_val) 

### Save the XBG model

In [ ]:
#Uncomment when actually running

# pre_saildrone now called utils, carry through this change
utils.save_model(model, model_output_dir, approach, ens, member)
print(datetime.datetime.now())
print(ens)
print(member)

## Test the XGB Model

### Preliminary Analysis on XBG Test Error Metrics

In [ ]:
# Calculate some test error metrics and store in a dictionary
# evaluate_test is a function from pre_saildrone. it includes MSE, MAE, bias etc

y_pred_test = model.predict(X_test)

test_performance[ens][member] = utils.evaluate_test(y_test, y_pred_test)
print(test_performance[ens][member])

In [ ]:
# Redo this analysis on the unseen data
y_pred_unseen = model.predict(df.loc[unseen_sel,features_sel].to_numpy())

y_unseen = df.loc[unseen_sel,target_sel].to_numpy().ravel()
unseen_performance[ens][member] = utils.evaluate_test(y_unseen, y_pred_unseen)
print(unseen_performance[ens][member])

## Create the reconstruction

In [ ]:
# Create the reconstruction and save it
# Jake calls it seen
# This should just be all SOCAT locations for all training years (not test years)
y_pred_seen = model.predict(X)

In [ ]:
# Full reconstruction 
df['pCO2_DIC_recon'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_recon']] = y_pred_unseen   # Not in a SOCAT location, not even in test year
df.loc[sel,['pCO2_DIC_recon']] = y_pred_seen

In [ ]:
# All time/locations not sampled by SOCAT
df['pCO2_DIC_nosocat'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_nosocat']] = y_pred_unseen
df.loc[sel,['pCO2_DIC_nosocat']] = np.nan

In [ ]:
# Only at time/locations of SOCAT sampling
df['pCO2_DIC_socat'] = np.nan
df.loc[unseen_sel,['pCO2_DIC_socat']] = np.nan
df.loc[sel,['pCO2_DIC_socat']] = y_pred_seen
     
df['pCO2_DIC'] = df['pCO2_pCO2T_diff']
             
#DS_recon = df[['net_mask','socat_mask','pCO2_DIC','pCO2_DIC_recon','pCO2_DIC_socat','pCO2_DIC_nosocat']].to_xarray()
DS_recon = df[['net_mask','combined_mask','pCO2_DIC','pCO2_DIC_recon','pCO2_DIC_socat','pCO2_DIC_nosocat']].to_xarray()

## Save reconstructions

In [ ]:
# Uncomment when actually running            
#pre_saildrone_thea.save_recon(DS_recon, recon_output_dir, approach, ens, member)   

## Save best parameters and performance metrics

In [ ]:
# Saving best parameters and performance metrics

approach_output_dir = f"{other_output_dir}/{approach}"
param_fname = f"{approach_output_dir}/{approach}_best_params_dict.pickle"
test_perform_fname = f"{approach_output_dir}/{approach}_test_performance_dict.pickle"
unseen_perform_fname = f"{approach_output_dir}/{approach}_unseen_performance_dict.pickle"

Path(approach_output_dir).mkdir(parents=True, exist_ok=True)

with open(param_fname, 'wb') as handle: #WHAT DOES wb MEAN
    pickle.dump(best_params, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open(test_perform_fname, 'wb') as handle:
    pickle.dump(test_performance, handle)
with open(unseen_perform_fname, 'wb') as handle:
    pickle.dump(unseen_performance, handle)
    
# Convert performance metrics to dataframes
test_df = pd.DataFrame.from_dict({(i,j): test_performance[i][j]
                                  for i in test_performance.keys()
                                  for j in test_performance[i].keys()},
                                 orient='index')

unseen_df = pd.DataFrame.from_dict({(i,j): unseen_performance[i][j]
                                  for i in unseen_performance.keys()
                                  for j in unseen_performance[i].keys()},
                                 orient='index')

test_df.index.names = ["model","member"]
unseen_df.index.names = ["model","member"]

# Save the dataframes too
test_df_fname = f"{approach_output_dir}/{approach}_test_performance_df.pickle"
unseen_df_fname = f"{approach_output_dir}/{approach}_unseen_performance_df.pickle"

test_df.to_pickle(test_df_fname)
unseen_df.to_pickle(unseen_df_fname)    

In [ ]:
#just checking what the saved trained datafiles look like 
test_2 = pd.read_pickle("/data/artemis/workspace/theimdal/saildrone/models/trained/xg/CESM/member_016/xg_model_pC02_2D_mon_CESM_016_1x1_198201-201701.joblib")

In [ ]:
test_2

In [ ]:
#checking out what the input data for the XGB looks like
#this table was generated in script 01
df